In [7]:
import re, os
import csv
import string 
import codecs

import pandas as pd
import numpy as np
from collections import defaultdict

import twokenize
import nltk
import unicodedata

In [86]:
sample = pd.read_csv("data/test/sample.csv")
print(sample.shape)
sample.head()

(2438, 3)


,item_id,item,label
0,'575735721744449536',Cyclone Pam hits New Zealand - dominion-post |...,Other Useful Information
1,'575736033058406400',#CylonePam devastates #Vanuatu: Regional disas...,Infrastructure and utilities
2,'575736037986578433',RT @9NewsAUS: Emergency response teams from Au...,Affected individuals
3,'575736076087644160',Vanuatu ‰_ #RT Cyclone Pam Lashes Vanuatu wit...,Other Useful Information
4,'575736131817369601',Aid agencies: Vanuatu conditions more challeng...,Other Useful Information


In [87]:
# quickly look into the lable values

class_label = sample.label.value_counts().rename('num_class').reset_index()
class_label['Percent'] = (class_label.num_class/raw.shape[0])*100
pd.DataFrame(class_label)

,index,num_class,Percent
0,Not related or irrelevant,718,23.790590
1,Other Useful Information,679,22.498343
2,Donations and volunteering,389,12.889331
3,Affected individuals,255,8.449304
4,Infrastructure and utilities,233,7.720345
5,Sympathy and support,164,5.434062


In [49]:
nepal = pd.read_csv("data/2015_Nepal_Earthquake_en/2015_nepal_eq_cf_labels.csv")
print(nepal.shape)
nepal.head()

(3018, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,871887237,False,finalized,3,1/31/2016 07:50:47,other_useful_information,1.0000,NaN,'591902695562170368',RT @PTI_News: Govt asks media houses to exerci...
1,871887238,False,finalized,3,1/31/2016 08:05:44,infrastructure_and_utilities_damage,0.6458,NaN,'591902695822331904',#money #news #top #b Magnitude-7.9 quake hits ...
2,871887239,False,finalized,3,1/31/2016 07:57:29,injured_or_dead_people,0.6939,NaN,'591902695943843840',RT @ABCNews24: #NepalEarthquake update: The In...
3,871887240,False,finalized,3,1/31/2016 08:21:07,missing_trapped_or_found_people,1.0000,NaN,'591902696371724288',RT @cnni: CNN obtains powerful images of baby ...
4,871887241,False,finalized,4,1/31/2016 07:57:23,sympathy_and_emotional_support,1.0000,NaN,'591902696375877632',My thoughts and prayers to everyone in Nepal.....


In [36]:
nepal1 = nepal[['tweet_id','tweet_text','choose_one_category']]
# nepal1 = nepal['event']='2015_nepal_eq'
nepal1.insert(3, 'event', '2015_nepal_eq')
nepal1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'591902695562170368',RT @PTI_News: Govt asks media houses to exerci...,other_useful_information,2015_nepal_eq
1,'591902695822331904',#money #news #top #b Magnitude-7.9 quake hits ...,infrastructure_and_utilities_damage,2015_nepal_eq
2,'591902695943843840',RT @ABCNews24: #NepalEarthquake update: The In...,injured_or_dead_people,2015_nepal_eq
3,'591902696371724288',RT @cnni: CNN obtains powerful images of baby ...,missing_trapped_or_found_people,2015_nepal_eq
4,'591902696375877632',My thoughts and prayers to everyone in Nepal.....,sympathy_and_emotional_support,2015_nepal_eq


In [50]:
pam = pd.read_csv("data/2015_Cyclone_Pam_en/2015_cyclone_pam_cf_labels.csv")
print(pam.shape)
pam.head()

(2013, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,871980227,False,finalized,3,1/31/2016 20:49:12,other_useful_information,1.0000,NaN,'575735721744449536',Cyclone Pam hits New Zealand - dominion-post |...
1,871980228,False,finalized,3,1/31/2016 21:43:52,infrastructure_and_utilities_damage,0.6667,NaN,'575736033058406400',#CylonePam devastates #Vanuatu: Regional disas...
2,871980229,False,finalized,4,1/31/2016 21:22:30,injured_or_dead_people,1.0000,NaN,'575736037986578433',RT @9NewsAUS: Emergency response teams from Au...
3,871980230,False,finalized,3,1/31/2016 21:20:00,other_useful_information,0.6765,NaN,'575736076087644160',Vanuatu ‰_ #RT Cyclone Pam Lashes Vanuatu wit...
4,871980231,False,finalized,3,1/31/2016 21:22:34,other_useful_information,1.0000,NaN,'575736131817369601',Aid agencies: Vanuatu conditions more challeng...


In [51]:
pam1 = pam[['tweet_id','tweet_text','choose_one_category']]
pam1.insert(3, 'event', '2015_Cyclone_Pam')
pam1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'575735721744449536',Cyclone Pam hits New Zealand - dominion-post |...,other_useful_information,2015_Cyclone_Pam
1,'575736033058406400',#CylonePam devastates #Vanuatu: Regional disas...,infrastructure_and_utilities_damage,2015_Cyclone_Pam
2,'575736037986578433',RT @9NewsAUS: Emergency response teams from Au...,injured_or_dead_people,2015_Cyclone_Pam
3,'575736076087644160',Vanuatu ‰_ #RT Cyclone Pam Lashes Vanuatu wit...,other_useful_information,2015_Cyclone_Pam
4,'575736131817369601',Aid agencies: Vanuatu conditions more challeng...,other_useful_information,2015_Cyclone_Pam


In [52]:
hagupit = pd.read_csv("data/2014_Philippines_Typhoon_Hagupit_en/2014_typhoon_hagupit_cf_labels.csv")
print(hagupit.shape)
hagupit.head()

(2013, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,872563518,False,finalized,3,2/2/2016 20:41:26,other_useful_information,1.0000,NaN,'540003786811793408',NASA sees Hagupit weaken to a depression enrou...
1,872563519,False,finalized,3,2/2/2016 19:24:50,not_related_or_irrelevant,0.3438,NaN,'540003796986773504',RT @ANCALERTS: Caloocan suspends classes in al...
2,872563520,False,finalized,3,2/2/2016 20:43:41,not_related_or_irrelevant,1.0000,NaN,'540003797351665664',TT WorldWide 05:17 1.#NEvsSD 2.#EatenAlive 3.#...
3,872563521,False,finalized,3,2/2/2016 20:49:07,caution_and_advice,0.6471,NaN,'540003870131232769',RT @annecurtissmith: Watching the news. Keep s...
4,872563522,False,finalized,3,2/2/2016 19:05:16,not_related_or_irrelevant,0.6571,NaN,'540003980076527619',Google inbox invite http://t.co/R3VC6pjOUA htt...


In [53]:
hagupit1 = hagupit[['tweet_id','tweet_text','choose_one_category']]
# nepal1 = nepal['event']='2015_nepal_eq'
hagupit1.insert(3, 'event', '2014_Philippines_Typhoon')
hagupit1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'540003786811793408',NASA sees Hagupit weaken to a depression enrou...,other_useful_information,2014_Philippines_Typhoon
1,'540003796986773504',RT @ANCALERTS: Caloocan suspends classes in al...,not_related_or_irrelevant,2014_Philippines_Typhoon
2,'540003797351665664',TT WorldWide 05:17 1.#NEvsSD 2.#EatenAlive 3.#...,not_related_or_irrelevant,2014_Philippines_Typhoon
3,'540003870131232769',RT @annecurtissmith: Watching the news. Keep s...,caution_and_advice,2014_Philippines_Typhoon
4,'540003980076527619',Google inbox invite http://t.co/R3VC6pjOUA htt...,not_related_or_irrelevant,2014_Philippines_Typhoon


In [54]:
cal = pd.read_csv("data/2014_California_Earthquake/2014_california_eq.csv")
print(cal.shape)
cal.head()

(2013, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,878953419,False,finalized,3,2/10/2016 12:47:18,other_useful_information,0.6875,NaN,'503642976422096896',RT @nicoleewayne: Tennessee USA Knoxville http...
1,878953420,False,finalized,3,2/10/2016 13:18:47,infrastructure_and_utilities_damage,0.7059,NaN,'503642976976113664',RT @SFGate: We're updating this interactive ma...
2,878953421,False,finalized,3,2/10/2016 12:05:43,injured_or_dead_people,1.0000,NaN,'503642977588494337',RT @YourAnonNews: Strong 6.1 Earthquake Rocks ...
3,878953422,False,finalized,3,2/10/2016 13:31:33,infrastructure_and_utilities_damage,0.6471,NaN,'503642979429388288',RT @heyyouapp: Wisconsin USA Madison http://t....
4,878953423,False,finalized,3,2/10/2016 12:47:53,other_useful_information,0.7027,NaN,'503642980881022976',"RT @scullather: ""@infodude: amazing use of #Bi..."


In [55]:
cal1 = cal[['tweet_id','tweet_text','choose_one_category']]
# nepal1 = nepal['event']='2015_nepal_eq'
cal1.insert(3, 'event', '2014_California_Earthquake')
cal1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'503642976422096896',RT @nicoleewayne: Tennessee USA Knoxville http...,other_useful_information,2014_California_Earthquake
1,'503642976976113664',RT @SFGate: We're updating this interactive ma...,infrastructure_and_utilities_damage,2014_California_Earthquake
2,'503642977588494337',RT @YourAnonNews: Strong 6.1 Earthquake Rocks ...,injured_or_dead_people,2014_California_Earthquake
3,'503642979429388288',RT @heyyouapp: Wisconsin USA Madison http://t....,infrastructure_and_utilities_damage,2014_California_Earthquake
4,'503642980881022976',"RT @scullather: ""@infodude: amazing use of #Bi...",other_useful_information,2014_California_Earthquake


In [56]:
pak = pd.read_csv("data/2013_Pakistan_eq/2013_pakistan_eq.csv")
print(pak.shape)
pak.head()

(2013, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,879819338,False,finalized,3,2/11/2016 11:41:40,other_useful_information,0.3429,NaN,'382813388503412736',USGS reports a M1.7 #earthquake 70km ESE of He...
1,879819339,False,finalized,3,2/11/2016 09:52:13,not_related_or_irrelevant,0.6345,NaN,'382813391435206659',"QuakeFactor M 3.0, Southern Alaska: Sunday, Se..."
2,879819340,False,finalized,3,2/11/2016 08:00:14,not_related_or_irrelevant,0.6757,NaN,'382813401845485568',What you wanna say abt #TTP #PeaceTalks &amp; ...
3,879819341,False,finalized,3,2/11/2016 09:36:47,donation_needs_or_offers_or_volunteering_services,0.6372,NaN,'382813437513834497',Top Story: Pakistan quake survivors face long ...
4,879819342,False,finalized,3,2/11/2016 07:37:44,injured_or_dead_people,1.0000,NaN,'382813440655376384',"RT @SaimaMohsin: 264 dead, more than 400 injur..."


In [57]:
pak1 = pak[['tweet_id','tweet_text','choose_one_category']]
pak1.insert(3, 'event', '2013_pakistan_eq')
pak1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'382813388503412736',USGS reports a M1.7 #earthquake 70km ESE of He...,other_useful_information,2013_pakistan_eq
1,'382813391435206659',"QuakeFactor M 3.0, Southern Alaska: Sunday, Se...",not_related_or_irrelevant,2013_pakistan_eq
2,'382813401845485568',What you wanna say abt #TTP #PeaceTalks &amp; ...,not_related_or_irrelevant,2013_pakistan_eq
3,'382813437513834497',Top Story: Pakistan quake survivors face long ...,donation_needs_or_offers_or_volunteering_services,2013_pakistan_eq
4,'382813440655376384',"RT @SaimaMohsin: 264 dead, more than 400 injur...",injured_or_dead_people,2013_pakistan_eq


In [59]:
india = pd.read_csv("data/2014_India_floods/2014_india_floods.csv")
print(india.shape)
india.head()

(2003, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,874983955,False,finalized,3,2/4/2016 13:10:28,injured_or_dead_people,1.0,NaN,'498497663609085955',"RT @timesofindia: Nepal flood toll hits 101, f..."
1,874983956,False,finalized,3,2/4/2016 11:31:02,injured_or_dead_people,1.0,NaN,'498497681036025856',"Deadly monsoon hits India, Nepal: Dozens of pe..."
2,874983957,False,finalized,3,2/4/2016 12:04:08,injured_or_dead_people,1.0,NaN,'498497696949235712',Many dead in Nepal and India floods: At least ...
3,874983958,False,finalized,3,2/4/2016 11:48:45,injured_or_dead_people,1.0,NaN,'498497699659137024',RT @ndtv: Flood fury in North India: dozens ki...
4,874983959,False,finalized,3,2/4/2016 13:37:12,injured_or_dead_people,1.0,NaN,'498497714183626754',"Flood Fury in North India: Dozens Killed, Lakh..."


In [60]:
india1 = india[['tweet_id','tweet_text','choose_one_category']]
india1.insert(3, 'event', '2014_india_floods')
india1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'498497663609085955',"RT @timesofindia: Nepal flood toll hits 101, f...",injured_or_dead_people,2014_india_floods
1,'498497681036025856',"Deadly monsoon hits India, Nepal: Dozens of pe...",injured_or_dead_people,2014_india_floods
2,'498497696949235712',Many dead in Nepal and India floods: At least ...,injured_or_dead_people,2014_india_floods
3,'498497699659137024',RT @ndtv: Flood fury in North India: dozens ki...,injured_or_dead_people,2014_india_floods
4,'498497714183626754',"Flood Fury in North India: Dozens Killed, Lakh...",injured_or_dead_people,2014_india_floods


In [25]:
mex = pd.read_csv("data/2014_Hurricane_Odile_Mexico_en/2014_hurricane_odile.csv")
print(mex.shape)
mex.head()

(2013, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,880160885,False,finalized,3,2/11/2016 20:15:22,infrastructure_and_utilities_damage,1.0000,NaN,'511423101150953472',RT @TLW3: Hurricane Odile Damages Update: Mexi...
1,880160886,False,finalized,3,2/11/2016 20:04:06,injured_or_dead_people,0.6944,NaN,'511423131605430272',"Coachella, keep it safe, remnants arriving soo..."
2,880160887,False,finalized,3,2/11/2016 19:45:49,infrastructure_and_utilities_damage,0.6579,NaN,'511423166283935744',BBC News - Hurricane Odile damages Mexico's Ba...
3,880160888,False,finalized,3,2/11/2016 19:36:19,donation_needs_or_offers_or_volunteering_services,0.7027,NaN,'511423208801984512',LET'S HELP REBUILD BCS DEVASTATED BY HURRICANE...
4,880160889,False,finalized,3,2/11/2016 19:51:49,infrastructure_and_utilities_damage,0.6579,NaN,'511423209158504448',"""In the wake of Hurricane Odileâ€™s destructio..."


In [61]:
mex1 = mex[['tweet_id','tweet_text','choose_one_category']]
mex1.insert(3, 'event', '2014_Hurricane_Odile_Mexico')
mex1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'511423101150953472',RT @TLW3: Hurricane Odile Damages Update: Mexi...,infrastructure_and_utilities_damage,2014_Hurricane_Odile_Mexico
1,'511423131605430272',"Coachella, keep it safe, remnants arriving soo...",injured_or_dead_people,2014_Hurricane_Odile_Mexico
2,'511423166283935744',BBC News - Hurricane Odile damages Mexico's Ba...,infrastructure_and_utilities_damage,2014_Hurricane_Odile_Mexico
3,'511423208801984512',LET'S HELP REBUILD BCS DEVASTATED BY HURRICANE...,donation_needs_or_offers_or_volunteering_services,2014_Hurricane_Odile_Mexico
4,'511423209158504448',"""In the wake of Hurricane Odileâ€™s destructio...",infrastructure_and_utilities_damage,2014_Hurricane_Odile_Mexico


In [62]:
cli = pd.read_csv("data/2014_Chile_Earthquake_en/2014_chile_eq_en.csv")
print(cli.shape)
cli.head()

(2013, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,871686722,False,finalized,3,1/30/2016 16:12:38,sympathy_and_emotional_support,0.6486,NaN,'451187618697736192',"#PrayForChile They've prayed for us, Philippin..."
1,871686723,False,finalized,3,1/30/2016 16:28:16,not_related_or_irrelevant,1.0000,NaN,'451187620945887232',I hope that @justinbieber will follow me.I'm s...
2,871686724,False,finalized,4,1/30/2016 16:10:14,sympathy_and_emotional_support,1.0000,NaN,'451187621843070976',RT @AvrilLavigne: Thinking of everyone in Chil...
3,871686725,False,finalized,3,1/30/2016 16:10:54,infrastructure_and_utilities_damage,0.6757,NaN,'451187621901791232',"Chile assesses damage after massive quake, tsu..."
4,871686726,False,finalized,3,1/30/2016 16:12:38,other_useful_information,0.6486,NaN,'451187622077947904',Disaster declared after Chile quake: Chile dec...


In [63]:
cli1 = cli[['tweet_id','tweet_text','choose_one_category']]
cli1.insert(3, 'event', '2014_Chile_Earthquake')
cli1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'451187618697736192',"#PrayForChile They've prayed for us, Philippin...",sympathy_and_emotional_support,2014_Chile_Earthquake
1,'451187620945887232',I hope that @justinbieber will follow me.I'm s...,not_related_or_irrelevant,2014_Chile_Earthquake
2,'451187621843070976',RT @AvrilLavigne: Thinking of everyone in Chil...,sympathy_and_emotional_support,2014_Chile_Earthquake
3,'451187621901791232',"Chile assesses damage after massive quake, tsu...",infrastructure_and_utilities_damage,2014_Chile_Earthquake
4,'451187622077947904',Disaster declared after Chile quake: Chile dec...,other_useful_information,2014_Chile_Earthquake


In [64]:
paki = pd.read_csv("data/2014_Pakistan_floods/2014_pakistan_floods_cf_labels.csv")
print(paki.shape)
paki.head()

(2013, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,876935612,False,finalized,3,2/7/2016 16:23:33,injured_or_dead_people,1.0000,NaN,'508332154271461376',RT @Endtimesnews: Raging Floods Kill Over 440 ...
1,876935613,False,finalized,3,2/7/2016 17:44:28,donation_needs_or_offers_or_volunteering_services,1.0000,NaN,'508332173532073985',RT @Joydas: Please use Kashmir Flood hashtag o...
2,876935614,False,finalized,3,2/7/2016 11:03:15,other_useful_information,0.6667,NaN,'508332177608568832',Jammu and Kashmir Floods: Yasin Malik Creates ...
3,876935615,False,finalized,3,2/7/2016 11:15:55,injured_or_dead_people,1.0000,NaN,'508332184613027840',Pakistan Flood Sinks Boat Carrying Wedding Par...
4,876935616,False,finalized,3,2/7/2016 10:58:12,donation_needs_or_offers_or_volunteering_services,0.6471,NaN,'508332187448401920',RT @KlasraRauf: Pungovt claims CMSS took 9 hel...


In [65]:
paki1 = paki[['tweet_id','tweet_text','choose_one_category']]
paki1.insert(3, 'event', '2014_pakistan_floods')
paki1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'508332154271461376',RT @Endtimesnews: Raging Floods Kill Over 440 ...,injured_or_dead_people,2014_pakistan_floods
1,'508332173532073985',RT @Joydas: Please use Kashmir Flood hashtag o...,donation_needs_or_offers_or_volunteering_services,2014_pakistan_floods
2,'508332177608568832',Jammu and Kashmir Floods: Yasin Malik Creates ...,other_useful_information,2014_pakistan_floods
3,'508332184613027840',Pakistan Flood Sinks Boat Carrying Wedding Par...,injured_or_dead_people,2014_pakistan_floods
4,'508332187448401920',RT @KlasraRauf: Pungovt claims CMSS took 9 hel...,donation_needs_or_offers_or_volunteering_services,2014_pakistan_floods


In [66]:
ebola = pd.read_csv("data/2014_ebola_cf/2014_ebola_virus.csv")
print(ebola.shape)
ebola.head()

(2017, 10)


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,choose_one_category,choose_one_category:confidence,choose_one_category_gold,tweet_id,tweet_text
0,916223154,False,finalized,6,3/24/2016 11:46:49,treatment,0.6867,NaN,'497086296385675264',RT @UNICEF: #Ebola symptoms. Early treatment m...
1,916223155,False,finalized,3,3/24/2016 11:05:11,not_related_or_irrelevant,1.0000,NaN,'497086297475805184',Thinking about telling leadership to make indi...
2,916223156,False,finalized,3,3/24/2016 12:35:48,other_useful_information,0.6876,NaN,'497086300781309952',"We know people are contracting the disease, an..."
3,916223157,False,finalized,3,3/24/2016 11:06:24,affected_people,0.6667,NaN,'497086305671864321',RT @gretawire: This is @foxnewsalert: Nurse #2...
4,916223158,False,finalized,4,3/24/2016 11:49:07,other_useful_information,0.7560,NaN,'497086306443202560',You Asked Questions About Ebola â€” And We Hav...


In [67]:
ebola1 = ebola[['tweet_id','tweet_text','choose_one_category']]
ebola1.insert(3, 'event', '2014_ebola')
ebola1.head()

,tweet_id,tweet_text,choose_one_category,event
0,'497086296385675264',RT @UNICEF: #Ebola symptoms. Early treatment m...,treatment,2014_ebola
1,'497086297475805184',Thinking about telling leadership to make indi...,not_related_or_irrelevant,2014_ebola
2,'497086300781309952',"We know people are contracting the disease, an...",other_useful_information,2014_ebola
3,'497086305671864321',RT @gretawire: This is @foxnewsalert: Nurse #2...,affected_people,2014_ebola
4,'497086306443202560',You Asked Questions About Ebola â€” And We Hav...,other_useful_information,2014_ebola


In [83]:
events = pd.concat([nepal1, pam1, hagupit1, cal1, pak1, india1, mex1, cli1, paki1])
print(events.shape)
events.head()

(19112, 4)


,tweet_id,tweet_text,choose_one_category,event
0,'591902695562170368',RT @PTI_News: Govt asks media houses to exerci...,other_useful_information,2015_nepal_eq
1,'591902695822331904',#money #news #top #b Magnitude-7.9 quake hits ...,infrastructure_and_utilities_damage,2015_nepal_eq
2,'591902695943843840',RT @ABCNews24: #NepalEarthquake update: The In...,injured_or_dead_people,2015_nepal_eq
3,'591902696371724288',RT @cnni: CNN obtains powerful images of baby ...,missing_trapped_or_found_people,2015_nepal_eq
4,'591902696375877632',My thoughts and prayers to everyone in Nepal.....,sympathy_and_emotional_support,2015_nepal_eq


In [82]:
# quickly look into the lable values

class_label = events.choose_one_category.value_counts().rename('num_class').reset_index()
class_label['Percent'] = (class_label.num_class/raw.shape[0])*100
pd.DataFrame(class_label)

,index,num_class,Percent
0,other_useful_information,5729,189.827700
1,donation_needs_or_offers_or_volunteering_services,2610,86.481113
2,injured_or_dead_people,2510,83.167661
3,not_related_or_irrelevant,2345,77.700464
4,sympathy_and_emotional_support,1969,65.241882
5,infrastructure_and_utilities_damage,1851,61.332008
6,caution_and_advice,1063,35.222001
7,displaced_people_and_evacuations,633,20.974155
8,missing_trapped_or_found_people,402,13.320080


In [84]:
events.columns = ['tweet_id', 'tweet_text', 'label', 'event']
events.head()

,tweet_id,tweet_text,label,event
0,'591902695562170368',RT @PTI_News: Govt asks media houses to exerci...,other_useful_information,2015_nepal_eq
1,'591902695822331904',#money #news #top #b Magnitude-7.9 quake hits ...,infrastructure_and_utilities_damage,2015_nepal_eq
2,'591902695943843840',RT @ABCNews24: #NepalEarthquake update: The In...,injured_or_dead_people,2015_nepal_eq
3,'591902696371724288',RT @cnni: CNN obtains powerful images of baby ...,missing_trapped_or_found_people,2015_nepal_eq
4,'591902696375877632',My thoughts and prayers to everyone in Nepal.....,sympathy_and_emotional_support,2015_nepal_eq


In [85]:
events.to_csv('events.csv', index = None, header=True, encoding='utf-8')